In [1]:
# Используем алгоритмы и код из Seminar3.ipynb для онкогенных тирозинкиназ (ABL1)

# Требования:
#   biopython, requests, clustalo (CLI), ete3

from Bio import Entrez, SeqIO, AlignIO, Phylo
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import subprocess

Entrez.email = "egor.masov@yandex.ru"


In [ ]:
# 1. BLAST: получить ≥5 гомологов ABL1 (UniProt P00519, PDB 1IEP)
# ---------------------------------------------------------------
# Скачиваем исходную последовательность ABL1
seq_handle = Entrez.efetch(db="protein", id="P00519", rettype="fasta", retmode="text")
query_record = SeqIO.read(seq_handle, "fasta")
seq_handle.close()
SeqIO.write(query_record, "ABL1_query.fasta", "fasta")

# Запускаем BLASTp
blast_handle = NCBIWWW.qblast("blastp", "nr", query_record.format("fasta"), hitlist_size=500)
blast_record = NCBIXML.read(blast_handle)

# Фильтруем гомологи по идентичности (10–100%) и покрытию ≥80%
homolog_ids = []
for alignment in blast_record.alignments:
    # print(alignment)
    for hsp in alignment.hsps:
        pid = 100 * hsp.identities / hsp.align_length
        cov = 100 * hsp.align_length / len(query_record.seq)
        if 10 <= pid <= 100 and cov >= 80:
            uid = alignment.hit_id.split("|")[3]  # UniProt или GI
            homolog_ids.append(uid)
    if len(homolog_ids) >= 5:
        break

# Сохраняем первые 5 гомологов
homolog_ids = homolog_ids[:5]
fasta_handle = Entrez.efetch(db="protein", id=",".join(homolog_ids), rettype="fasta", retmode="text")
SeqIO.write(SeqIO.parse(fasta_handle, "fasta"), "ABL1_homologs.fasta", "fasta")
fasta_handle.close()

In [ ]:
# 2. ClustalO: множественное выравнивание + percent identity matrix
# ------------------------------------------------------------------
clustalo_cmd = [
    "clustalo",
    "-i", "ABL1_homologs.fasta",
    "-o", "ABL1_alignment.clustal",
    "--percent-id",
    "--distmat-out=ABL1_percent_identity.tsv",
    "--force"
]
subprocess.run(clustalo_cmd, check=True)

# Конвертация выравнивания для Phylo
AlignIO.convert("ABL1_alignment.clustal", "clustal", "ABL1_alignment.phy", "phylip")

In [ ]:
# 3. Построение деревьев UPGMA и NJ и их визуализация
# ---------------------------------------------------
alignment = AlignIO.read("ABL1_alignment.clustal", "clustal")
calculator = DistanceCalculator("identity")
dist_matrix = calculator.get_distance(alignment)

constructor = DistanceTreeConstructor()
tree_upgma = constructor.upgma(dist_matrix)
tree_nj   = constructor.nj(dist_matrix)

# Сохраняем деревья в PhyloXML
Phylo.write(tree_upgma, "ABL1_tree_UPGMA.xml", "phyloxml")
Phylo.write(tree_nj,    "ABL1_tree_NJ.xml",    "phyloxml")

# Визуализация через ETE3
from ete3 import Tree, TreeStyle

for name, xml in [("UPGMA", "ABL1_tree_UPGMA.xml"), ("NJ", "ABL1_tree_NJ.xml")]:
    t = Tree(xml, format=1)
    ts = TreeStyle()
    ts.show_branch_support = True
    t.render(f"ABL1_{name}_tree.png", tree_style=ts)

print("Файлы для отчета:")
print("- Uniprot ID исходного белка: P00519 (ABL1); PDB ID: 1IEP")
print("- Множественное выравнивание: ABL1_alignment.clustal")
print("- Percent identity matrix: ABL1_percent_identity.tsv")
print("- Деревья: ABL1_UPGMA_tree.png, ABL1_NJ_tree.png")